In [82]:
import argparse
import json
import os
import time
import numpy as np
from tqdm import tqdm

#Add openai key here
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [37]:
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

In [38]:
questions = []
questions_path = '/root/LLaVA-Recipe/llava/eval/table/question_recipe_1k_3q.jsonl'
with open(questions_path, 'r') as file:
    for lines in file:
        data = json.loads(lines)
        questions.append(data)
print(questions)
print(len(questions))

[{'image': '7a4833e18e.jpg', 'question_id': 1, 'text': 'What recipe can be made with the items in this image?', 'category': 'generic'}, {'image': '7a4833e18e.jpg', 'question_id': 2, 'text': 'How should the food in this picture be prepared?', 'category': 'generic'}, {'image': '7a4833e18e.jpg', 'question_id': 3, 'text': 'What cooking method is used for the food shown in this image?', 'category': 'generic'}, {'image': '59434caadc.jpg', 'question_id': 1, 'text': 'What recipe can be made with the items in this image?', 'category': 'generic'}, {'image': '59434caadc.jpg', 'question_id': 2, 'text': 'How should the food in this picture be prepared?', 'category': 'generic'}, {'image': '59434caadc.jpg', 'question_id': 3, 'text': 'What cooking method is used for the food shown in this image?', 'category': 'generic'}, {'image': 'a75a21cb56.jpg', 'question_id': 1, 'text': 'What recipe can be made with the items in this image?', 'category': 'generic'}, {'image': 'a75a21cb56.jpg', 'question_id': 2, 't

In [39]:
answer1 = []
answer1_path = '/root/LLaVA-Recipe/llava/eval/table/answer/answer-7b-IIM-e1.jsonl'
with open(answer1_path, 'r') as file:
    for lines in file:
        data = json.loads(lines)
        answer1.append(data)
print(answer1)
print(len(answer1))

[{'image': '7a4833e18e.jpg', 'question_id': 1, 'prompt': 'What recipe can be made with the items in this image?', 'text': 'The image shows a combination of ingredients that can be used to create a delightful treat. You can use these ingredients to make a delicious cupcake with a unique twist.', 'answer_id': 'QrtY4HUY7yBXsqq64ENrnp', 'model_id': 'llava-lora-7b-10K-IIM-e1', 'metadata': {}}, {'image': '7a4833e18e.jpg', 'question_id': 2, 'prompt': 'How should the food in this picture be prepared?', 'text': 'In the image, the food is prepared by combining the sugar, flour, baking powder, salt, and cinnamon in a bowl. Then, butter and egg are added to the mixture and mixed until smooth. The batter is then divided into 12 muffin cups and baked in the oven until golden brown.', 'answer_id': 'H2tAhuvKZmJtfr5kLJwiGb', 'model_id': 'llava-lora-7b-10K-IIM-e1', 'metadata': {}}, {'image': '7a4833e18e.jpg', 'question_id': 3, 'prompt': 'What cooking method is used for the food shown in this image?', 't

In [61]:
answer2 = []
answer2_path = '/root/LLaVA-Recipe/llava/eval/table/answer/answer_llava-7b.jsonl'
with open(answer2_path, 'r') as file:
    for lines in file:
        data = json.loads(lines)
        answer2.append(data)
print(answer2)
print(len(answer2))

[{'image': '7a4833e18e.jpg', 'question_id': 1, 'prompt': 'What recipe can be made with the items in this image?', 'text': 'The image features a cupcake with a blackberry on top, sitting on a blue table. To make a cupcake with a blackberry on top, you would need the following ingredients: cupcake batter, blackberries, powdered sugar, and a cupcake liner. You can either bake the cupcake batter in a cupcake pan or use a muffin tin to create the cupcake. Once the cupcake is baked, you can remove it from the pan and let it cool. Then, you can top the cupcake with a blackberry, and sprinkle powdered sugar on top. Finally, you can place the cupcake in a cupcake liner and serve it. This recipe is a simple and delicious way to enjoy a cupcake with a touch of fruit flavor.', 'answer_id': 'VuikWmHYQo7kZBDoqVRhbL', 'model_id': 'llava-v1.5-7b', 'metadata': {}}, {'image': '7a4833e18e.jpg', 'question_id': 2, 'prompt': 'How should the food in this picture be prepared?', 'text': 'In the image, there is

In [41]:
role = 'Assistant'
prompt = "We would like to request your feedback on the performance of two AI assistants in response to the user question displayed later.\nPlease rate the helpfulness, relevance, accuracy, level of details of their responses. Each assistant receives an overall score on a scale of 1 to 10, where a higher score indicates better overall performance.\nPlease first output a single line containing only two values indicating the scores for Assistant 1 and 2, respectively. The two scores are separated by a space.\nIn the subsequent line, please provide a comprehensive explanation of your evaluation, avoiding any potential bias and ensuring that the order in which the responses were presented does not affect your judgment."

In [69]:
contexts = []
contexts_path = '/root/LLaVA-Recipe/dataset/recipe1m/test_label/test1K.json'
with open(contexts_path, 'r') as file:
    all_data = json.load(file)
    for data in all_data:
        context = 'dish name: ' + data['title'] + '.\n' + 'Ingredients: ' + data['ingredients'] + '.\n' + 'Instructions: ' + data['instructions'] + '.'
        contexts.append(context)
print(contexts)
print(len(contexts))
print(contexts[0])

['dish name: Wild Blackberry Vanilla Cupcakes.\nIngredients: 1 cup Unsalted Butter, Room Temperature,1-23 cup Granulated Sugar,4 whole Eggs,1 Tablespoon Vanilla Extract,3 cups All-purpose Flour,1 Tablespoon Baking Powder,1/2 teaspoons Baking Soda,1- 1/4 cup Buttermilk,2 cups Blackberries,2 cups (or As Needed) Buttercream Icing, Your Preference,Sanding Sugar, As Needed.\nInstructions: Preheat oven to 350 degrees F (180 degrees C). In a mixer bowl fitted with a paddle attachment, beat butter with sugar until light and fluffy (a few minutes). Beat in eggs, one at a time, beating well after each addition. Beat in vanilla. Whisk together flour, baking powder, baking soda and salt. Stir dry mixture into wet (butter) mixture alternately with buttermilk, making 3 additions of dry ingredients and 3 of buttermilk. Fold in blackberries. Using an ice cream scoop or 1/4-cup measurer, scoop batter into a muffin tin lined with cupcake liners. Bake for approximately 15-20 minutes, until cake tester (t

In [43]:
NUM_SECONDS_TO_SLEEP = 0.5

def get_eval(content: str, max_tokens: int):
    while True:
        try:
            response = client.chat.completions.create(
                model='gpt-3.5-turbo',
                messages=[{
                    'role': 'system',
                    'content': 'You are a helpful and precise assistant for checking the quality of the answer.'
                }, {
                    'role': 'user',
                    'content': content,
                }],
                temperature=0.2,  # TODO: figure out which temperature is best for evaluation
                max_tokens=max_tokens,
            )
            break
        # except client.chat.error.RateLimitError:
        #     pass
        except Exception as e:
            print(e)
        time.sleep(NUM_SECONDS_TO_SLEEP)

    return response.choices[0].message.content

In [44]:
def parse_score(review):
    try:
        score_pair = review.split('\n')[0]
        score_pair = score_pair.replace(',', ' ')
        sp = score_pair.split(' ')
        if len(sp) == 2:
            return [float(sp[0]), float(sp[1])]
        else:
            print('error', review)
            return [-1, -1]
    except Exception as e:
        print(e)
        print('error', review)
        return [-1, -1]

In [91]:
reviews = []
num_eval = 300
for ques, ans1, ans2, context in tqdm(zip(questions[:num_eval], answer1[:num_eval], answer2[:num_eval], contexts[:num_eval]), total=num_eval, desc="Evaluating"):
    content = (f'[Context]\n{context}\n\n'
               f'[Question]\n{ques["text"]}\n\n'
               f'[{role} 2]\n{ans2["text"]}\n\n[End of {role} 2]\n\n'
               f'[{role} 1]\n{ans1["text"]}\n\n[End of {role} 1]\n\n'
               f'[System]\n{prompt}\n\n'
               )
    review = get_eval(content, max_tokens=1024)
    reviews.append(review)
    # print(review)
print(len(reviews))

Evaluating: 100%|██████████| 300/300 [17:25<00:00,  3.48s/it]

300


In [55]:
scores_cache = []

['7 6\nBoth Assistant 1 and Assistant 2 provided relevant responses by acknowledging that the ingredients in the image can be used to make a cupcake. However, they both lacked specific details and did not mention the key elements of the recipe provided in the context, such as the flavor (Wild Blackberry Vanilla Cupcakes), the baking instructions, and the icing recommendation. Assistant 1\'s response was slightly more helpful as it mentioned a "unique twist," which could imply the inclusion of blackberries in the cupcakes. Assistant 2\'s response was also relevant but lacked any additional insights or details. Overall, both responses could be improved by including more specific information from the context to enhance accuracy and helpfulness.', '6 6\nBoth Assistant 1 and Assistant 2 provided responses that are somewhat relevant to the question asked, as they both mention a baking process involving sugar, flour, butter, and eggs. However, the responses lack accuracy and detail in address

In [92]:
scores = []
for review in reviews:
    score = parse_score(review)
    scores.append(score)
scores = np.array(scores)
avg = np.mean(scores, axis=0)
print(f"Ours: {avg[0]} / 10 \nLLaVA-7B: {avg[1]} / 10")

Ours: 7.04 / 10 
LLaVA-7B: 6.173333333333333 / 10
